# Imports

In [ ]:
### Necessary imports ###
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#import tensorflow as tf
import sklearn as sk
#import matplotlib as mpl
#import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier as RF
import lightgbm as lgb
from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import PolynomialFeatures
from catboost import Pool,CatBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import svm, model_selection,tree, linear_model, neighbors, naive_bayes, ensemble 
from sklearn import discriminant_analysis, gaussian_process

from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector


from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV

## IML Funktionen

In [ ]:
from sklearn import tree
from sklearn.inspection import permutation_importance

def print_importance(clf):
    #get classifier per label(j)
    j = 0
    for rscv in clf.classifiers_:
        
        #get and sort importances from best estimator
        importances = rscv.best_estimator_.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        #plot_my_tree(rscv)
        # Print the feature ranking
        print("label " + str(test_y.columns[j]) + ":")

        for f in range(train_X.shape[1]-1):
            print("%d. %s (%f)" % (f +1 , feature_namen[indices[f]], importances[indices[f]]))
        j+= 1
        print()        

def plot_my_tree(rscv):
    tree1 = rscv.best_estimator_.estimators_[0].tree_
    fig = plt.figure(figsize=(20,15))
    _ = tree.plot_tree(rscv.best_estimator_.estimators_[0],
                  feature_names=feature_namen,
                  class_names=labels,
                  filled=True,
                  max_depth=3,
                  fontsize=10)        
        
def print_my_trees(clf):
    j=0
    for rscv in clf.classifiers_:
        print("label " + str(test_y.columns[j]) + ":")
        plot_my_tree(rscv)
        j+=1
        print()    

def print_permutation(clf):
    j=0 
    for rscv in clf.classifiers_:
        print("label " + str(test_y.columns[j]) + ":")
        j=0 
        r = permutation_importance(rscv.best_estimator_, test_X, test_y[test_y.columns[j]],
                              n_repeats=30,
                              random_state=0)

        for i in r.importances_mean.argsort()[::-1]:
            if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
                print(f"{feature_namen[i]:<8}   "
                    f"{r.importances_mean[i]:.3f}"
                    f" +/- {r.importances_std[i]:.3f}")
        j +=1
        print()
  
                                   
def print_coef(clf): 
    for rscv in clf.classifiers_:
        j=0        
        for array in rscv.coef_:
            #safe feature in array
            coef_features = []
            for number in array:
                coef_features.append([number, feature_namen[j]])
                j += 1
            coef_features.sort()
            for pair in coef_features[:8]:
                print(pair)
        print()
        j=1
                                   
def print_scores(clf):
    j= 0
    for rscv in clf.classifiers_:
        print("label " + str(test_y.columns[j]) + ":")
        #print(rscv.scores_.info())
        print(rscv.best_score_)
        print(rscv.best_params_)
        j+=1
        print()

# Transform-Funktion

In [ ]:
def transform_test(d):
    # Löschen der Werte mit zu vielen Null-Stellen und der ID
    d.drop(['respondent_id','health_insurance','employment_industry','employment_occupation', 'census_msa', 'hhs_geo_region', 'education', 'income_poverty'],axis=1,inplace = True)
    
    # All the NULL values are populated with the mode
    str_cols = d.select_dtypes(include = 'object').columns

    ### LabelEcoding all categorical types #####
    for col in d.columns:
        if d[col].isnull().sum() and d[col].dtypes != 'object':
            d[col].loc[(d[col].isnull())] = d[col].median()
    for col in d.columns:
        if d[col].isnull().sum() and d[col].dtypes == 'object':
            d[col].loc[(d[col].isnull())] = d[col].mode().max()
    

    
    #One hot encoder
    d = pd.get_dummies(d,drop_first=False)
    
    data = d
    ### Synthesizing two new features cleanliness level of the individual and opinion of vaccine ####    
    data['old'] = data['age_group_55 - 64 Years'] + data['age_group_65+ Years']
   
    data['cleanliness'] =  data['behavioral_antiviral_meds']+ data['behavioral_avoidance']+\
                        data['behavioral_face_mask']+data['behavioral_wash_hands']+\
                       data['behavioral_large_gatherings'] + data['behavioral_outside_home']+ data['behavioral_touch_face']
    data['opinion_h1n1'] = data['opinion_h1n1_vacc_effective'] + data['opinion_h1n1_risk']-\
                      data['opinion_h1n1_sick_from_vacc'] 
    data['opinion_seasonal'] = data['opinion_seas_vacc_effective']+\
                      data['opinion_seas_risk'] - data['opinion_seas_sick_from_vacc']
    data['good_opinion_vacc'] = np.where(data['opinion_seas_vacc_effective'] >= 3,1,0) # 5 before
    data['good_knowledge'] = np.where(data['h1n1_knowledge'] == 2,1,0)
    data['risk'] = np.where(data['opinion_h1n1_risk']>=4,1,0)
    data['old_recc_h1n1'] = data['age_group_55 - 64 Years'] + data['age_group_65+ Years']+\
                  data['doctor_recc_h1n1']
    data['old_recc_seasonal'] = data['age_group_55 - 64 Years'] + data['age_group_65+ Years']+\
                  data['doctor_recc_seasonal']
    
    data['risk'] = np.where(data['opinion_h1n1_risk']>=4,1,0)
    data['old'] = np.where(data['old_recc_h1n1']>=2,1,0)

  

    
    #data['a^2'] = data['age_group']*data['age_group']
    
    ###### Dropping other features #########
    data.drop([
        # droping features
        #'h1n1_knowledge', 'h1n1_concern',
        
        #'behavioral_antiviral_meds', 'behavioral_face_mask', 'behavioral_avoidance',
        #'behavioral_wash_hands','behavioral_large_gatherings', 'behavioral_outside_home',
        ##'behavioral_touch_face',
        
        #'education_< 12 Years', 'income_poverty_> $75,000',
        
        'opinion_h1n1_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc',
        'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_h1n1_risk',
        #'age_group'
        #'sex', 'chronic_med_condition', 'education', 'income_poverty','rent_or_own',
        #'employment_status','race', 'household_children', 'household_adults', 'child_under_6_months',
        #'marital_status',
        
        #'census_msa', 'hhs_geo_region'   
        
         ],axis=1,inplace = True)
    LE = LabelEncoder()
    for col in d.columns:
        data[col] = LE.fit_transform(d[col])
    return data

def transform_test(d):
    # Löschen der Werte mit zu vielen Null-Stellen und der ID
    d.drop(['respondent_id','health_insurance','employment_industry','employment_occupation', 'census_msa', 'hhs_geo_region', 'education', 'income_poverty' ],axis=1,inplace = True)
    
    # All the NULL values are populated with the mode
    str_cols = d.select_dtypes(include = 'object').columns

    ### LabelEcoding all categorical types #####
    for col in d.columns:
        if d[col].isnull().sum() and d[col].dtypes != 'object':
            d[col].loc[(d[col].isnull())] = d[col].median()
    for col in d.columns:
        if d[col].isnull().sum() and d[col].dtypes == 'object':
            d[col].loc[(d[col].isnull())] = d[col].mode().max()
    
    #LE = LabelEncoder()
    #for col in d.columns:
    #    d[col] = LE.fit_transform(d[col])
    
    #One hot encoder
    d = pd.get_dummies(d,drop_first=True)
    
    data = d
    ### Synthesizing two new features cleanliness level of the individual and opinion of vaccine ####
    data['opinion'] = data['opinion_h1n1_vacc_effective'] + data['opinion_h1n1_risk']+\
                  data['opinion_h1n1_sick_from_vacc'] + data['opinion_seas_vacc_effective']+\
                  data['opinion_seas_risk'] + data['opinion_seas_sick_from_vacc']
    data['cleanliness'] =  data['behavioral_antiviral_meds']+ data['behavioral_avoidance']+\
                        data['behavioral_face_mask']+data['behavioral_wash_hands']+\
                       data['behavioral_large_gatherings'] + data['behavioral_outside_home']+\
                       data['behavioral_touch_face']
    data['opinion_h1n1'] = data['opinion_h1n1_vacc_effective'] + data['opinion_h1n1_risk']-\
                      data['opinion_h1n1_sick_from_vacc'] 
    data['opinion_seasonal'] = data['opinion_seas_vacc_effective']+\
                      data['opinion_seas_risk'] - data['opinion_seas_sick_from_vacc']

    data['concern'] = np.where(data['h1n1_concern']>=2,1,0)
    data['good_opinion_vacc'] = np.where(data['opinion_seas_vacc_effective'] == 3,1,0) # 5 before
    data['good_knowledge'] = np.where(data['h1n1_knowledge'] == 2,1,0)
    data['risk'] = np.where(data['opinion_h1n1_risk']>=4,1,0)
    data['concern_knowledge'] = data['h1n1_concern']+data['h1n1_knowledge']
    
    #data['a^2'] = data['age_group']*data['age_group']
    
    ###### Dropping other features #########
    data.drop([
        # droping features
        'h1n1_knowledge', 'h1n1_concern',
        
        'behavioral_antiviral_meds', 'behavioral_face_mask', 'behavioral_avoidance',
        'behavioral_wash_hands','behavioral_large_gatherings', 'behavioral_outside_home',
        'behavioral_touch_face',
        
        #'education_< 12 Years', 'income_poverty_> $75,000',
        
        'opinion_h1n1_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc',
        'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_h1n1_risk'
        
        #'sex', 'chronic_med_condition','age_group', 'education', 'income_poverty','rent_or_own',
        #'employment_status','race', 'household_children', 'household_adults', 'child_under_6_months',
        #'marital_status',
        
        #'census_msa', 'hhs_geo_region'   
        
         ],axis=1,inplace = True)
    return data

## Import und vorbereiten der Daten

In [ ]:
#X = pd.read_csv(r'../input/changed-feature-names-flus-hot-prediction/training_set_090421.csv')
#read data from csv
X = pd.read_csv(r'../input/flu-shot-learning-h1n1-seasonal-flu-vaccines/training_set_features.csv')
Y = pd.read_csv(r'../input/flu-shot-learning-h1n1-seasonal-flu-vaccines/training_set_labels.csv')
#test_x_data = pd.read_csv(r'../input/flu-shot-learning-h1n1-seasonal-flu-vaccines/test_set_features.csv')
# 
# = Y
#Z.drop('respondent_id',axis = 1)
frames = [X,Y]
data = pd.concat(frames,axis=1)
#print(data.head())

train = transform_test(data)
label = train[['h1n1_vaccine','seasonal_vaccine']]
train.drop(['h1n1_vaccine','seasonal_vaccine'],axis=1,inplace = True)
frames =[train,label]
train = pd.concat(frames,axis=1)
features = train.columns[:-2]
labels = ['h1n1_vaccine', 'seasonal_vaccine']
feature_namen = train.columns.array

train,test = train_test_split(train,test_size = 0.2,shuffle = True)
train_X,train_y = train[features],train[labels]
test_X,test_y = test[features],test[labels]
train_X.head()



In [ ]:
n_categorical_features = (train_X.dtypes == 'category').sum()
n_numerical_features = (train_X.dtypes == 'float').sum()
print(f"Number of samples: {train_X.shape[0]}")
print(f"Number of features: {train_X.shape[1]}")
print(f"Number of categorical features: {n_categorical_features}")
print(f"Number of numerical features: {n_numerical_features}")


In [ ]:
#### Utility Function to print the accuracy of the model ###
def find_acc(clf,X,truth,s):
    pred = clf.predict_proba(X)
    pred = pred.toarray()
    accuracy = roc_auc_score(truth,pred)
    print(s+" Accuracy is : ",accuracy*100)

In [ ]:
#### Random Forest Classifier with Randomized Search CV for HPO #####

n_estimators = [100,200,500]
max_features = ['auto', 'sqrt']
max_depth = [15,20,25]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = XGBClassifier(feature_names = train_X,verbose = False)
etc = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20,\
                               cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
### Training & Performance ###
clf = BinaryRelevance(classifier=etc, require_dense=[True,True])
clf.fit(train_X,train_y)
find_acc(clf,train_X,train_y,'Training')
find_acc(clf,test_X,test_y,'Test')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [400,500,600]
max_features = ['auto', 'sqrt']
max_depth = [5,15,20]
min_samples_split = [2, 5, 10]
min_samples_leaf = [2, 3, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
RF=RandomForestClassifier()
rf = RF
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20,\
                               cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
### Training & Performance ###
clf = BinaryRelevance(classifier=rf_random, require_dense=[True,True])
clf.fit(train_X,train_y)
find_acc(clf,train_X,train_y,'Training')
find_acc(clf,test_X,test_y,'Test')
print_scores(clf)

# IML Auswertung des Classifiers

In [ ]:
print_importance(clf)

In [ ]:
print_my_trees(clf)

In [ ]:
print_permutation(clf)

In [ ]:
# test LogisticRegressionCV
clf = LogisticRegressionCV(cv=10, random_state=0)
clf = BinaryRelevance(classifier=clf, require_dense=[True,True])
clf.fit(train_X,train_y)
find_acc(clf,train_X,train_y,'Training')
find_acc(clf,test_X,test_y,'Test')

In [ ]:
print_coef(clf)

In [ ]:
print_permutation(clf)